In [100]:
import owlready2
# imports
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import OWL, RDF, RDFS, XSD
from owlready2 import *


In [101]:
NAMESPACE = "https://jaron@sprute.com/uni/actionable-knowledge-representation/feinschmecker"
onto = get_ontology(NAMESPACE)

def ThingFactory(name, BaseClass=Thing):
    with onto:
        return type(name, (BaseClass,), {})

def RelationFactory(name, domain:list[ThingClass]=None, range:list[ThingClass]=None):
    if domain is None:
        domain = [Thing]
    if range is None:
        range = [Thing]
    with onto:
        return type(name, (ObjectProperty, ), {
            "domain": domain,
            "range": range,
        })

Text = ThingFactory("Text")
Hello = ThingFactory("Hello", BaseClass=Text)
World = ThingFactory("World", BaseClass=Text)
continuesWith = RelationFactory("continuesWith", domain=[Text], range=[Text])

onto.destroy()
